# Openshift CLI Lab
In this lab, you will use the OpenShift Command Line (oc command) to manage applications, resources and the cluster itself. While many tasks can be performed through the OpenShift web console (GUI), the CLI offers powerful automation capabilities and direct access for advanced operations.

The oc command has been installed on your jumphosts.  The application is obtained from RedHat



This lab is designed to provide hands-on experience with the `oc` CLI. We will cover a range of essential commands to perform tasks similar to those you might have encountered in OpenShift GUI-based labs, including:

* Interacting with projects (namespaces)
* Deploying and managing applications
* Inspecting cluster resources

> During this lab the output you see will be the same as previously explored in the GUI


## Code Parameters 
!!! Before running the code below, ensure you have the correct parameters set for your environment. 

Change these parameters to match your environment

The following is actually a Python variable that is used when shell scripts are run.  Remember the Kernel is Python

In [ ]:
### Set Environment ###
student_number = "##"      # Replace with your student number

## Login to Openshift Cluster ##
- You may also need to change to your own project/namespace that you used in the previous lab

In [ ]:
### Login to OpenShift ###
!oc login -u s$student_number -p'!@34QWer' https://api.ocp.ucsx.hl.dns:6443 --insecure-skip-tls-verify


#### Expected Output 
The output should look similar to 

---

```
WARNING: Using insecure TLS client config. Setting this option is not supported!

Console URL: https://api.ocp.ucsx.hl.dns:6443/console
Authentication required for https://api.ocp.ucsx.hl.dns:6443 (openshift)
Username: s01
Password:
Login successful.
```

---

## Change Namespace
When you first login into the cluster via the oc command unless you specify otherwise, you are placed into the default namespace

Each student has been allocated a individual project with int OpenShift.  This is the same as the Namespace concept in native K8s

In [ ]:
### Change to your Project ###
!oc project ai-s$student_number

#### Expected Output 
The output should look similar to 

---

```
Now using project "ai-s01" on server "https://api.ocp.ucsx.hl.dns:6443".
```

---

## Explore the cluster
### Show nodes in cluster
* This shows the nodes in the cluster and their status.

In [ ]:
!oc get nodes

#### Expected Output 
The output should look similar to 

---

```
NAME               STATUS   ROLES                  AGE   VERSION
ocp1.ucsx.hl.dns   Ready    control-plane,master   57d   v1.31.7
ocp2.ucsx.hl.dns   Ready    control-plane,master   57d   v1.31.7 
ocp3.ucsx.hl.dns   Ready    control-plane,master   57d   v1.31.7 
ocp4.ucsx.hl.dns   Ready    worker                 57d   v1.31.7 
ocp5.ucsx.hl.dns   Ready    worker                 57d   v1.31.7
ocp6.ucsx.hl.dns   Ready    worker                 57d   v1.31.7
```
---

### Node details
Other than the ability to view a list of the nodes you can also look at the details of individual nodes


In [ ]:
### Check GPU Resources on Cluster Nodes ###
# The cluster currently has one physical GPU on node ocp4, but GPU time-slicing has
# been configured through the NVIDIA GPU Operator to create 8 virtual GPU slices
!oc describe node ocp4 | grep -A 6 Capacity

#### Expected Output 
The output should look similar to 

---

```
Capacity:
  cpu:                152
  ephemeral-storage:  204222Mi
  hugepages-1Gi:      0
  hugepages-2Mi:      0
  memory:             1056450804Ki
  nvidia.com/gpu:     8
```

---

In [ ]:
#### Check the number of allocated GPUs ###
!oc describe node ocp4 | grep -A 9 "Allocated resources"

#### Expected Output 
The output should look similar to 

---

```
Allocated resources:
  (Total limits may be over 100 percent, i.e., overcommitted.)
  Resource           Requests      Limits
  --------           --------      ------
  cpu                11349m (7%)   19210m (12%)
  memory             34880Mi (3%)  55100Mi (5%)
  ephemeral-storage  0 (0%)        0 (0%)
  hugepages-1Gi      0 (0%)        0 (0%)
  hugepages-2Mi      0 (0%)        0 (0%)
  nvidia.com/gpu     3             3
  ```

  ---
  

## Explore Deployed Application

- This includes pods, services, deployments, and more.
- Here you can see the
    - demo-manifest application is running.
    - The service is exposing the application on port 8080.
    - The route is exposing the application to the outside world.

In [ ]:
### Look at the deployed Pods ###
# Here you can see that two pod are running and one pod is in the "Completed" state. 
# The "Completed" pod is the build container that was used to build the container image from the git source code.
!oc get pods

#### Expected Output 
The output should look similar to 

---

```
NAME                                           READY   STATUS      RESTARTS   AGE
ai-open-webui-deployment-01-86bd8c545f-4rv8b   1/1     Running     0          18h
housley-counter-1-build                        0/1     Completed   0          3h42m
housley-counter-69fffdf548-7m5cp               1/1     Running     0          3h42m
```

---


In [ ]:
### Look at the deployed Services ###
# The service is used to expose the application internally within the cluster.
!oc get services

#### Expected Output 
The output should look similar to 

---

```
NAME                       TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)    AGE
ai-open-webui-service-01   ClusterIP   172.30.198.88   <none>        8080/TCP   18h
housley-counter            ClusterIP   172.30.203.60   <none>        8080/TCP   3h43m
```

---


In [ ]:
### Look at the deployed Routes ###
# The route is used to expose the application externally
# Here you can see the DNS name that can be used to access the application from outside the cluster.
!oc get routes

#### Expected Output 
The output should look similar to 

---

```
NAME                 HOST/PORT                                        PATH   SERVICES                   PORT       TERMINATION     WILDCARD
housley-counter      housley-counter-ai-s01.apps.ocp.ucsx.hl.dns             housley-counter            8080-tcp   edge/Redirect   None
openwebui-route-01   openwebui-route-01-ai-s01.apps.ocp.ucsx.hl.dns          ai-open-webui-service-01   8080       edge            None
```

---


### View all objects in a given project 
You will be currently positioned in your student namespace/project

You can query all objects that exist in that in that project

When you run the oc command you can specify the format in which the responses are return

Other key options include:
 - name
 - json
 - wide
 - yaml

Customised output can be created:
- custom-columns
- template

In [ ]:
#### Look at all resources in the project ###
!oc get all

#### Expected Output 
The output will be different to the following 

---

```
Warning: apps.openshift.io/v1 DeploymentConfig is deprecated in v4.14+, unavailable in v4.10000+
NAME                                               READY   STATUS      RESTARTS   AGE
pod/ai-open-webui-deployment-01-86bd8c545f-4rv8b   1/1     Running     0          18h
pod/housley-counter-1-build                        0/1     Completed   0          3h44m
pod/housley-counter-69fffdf548-7m5cp               1/1     Running     0          3h44m

NAME                               TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)    AGE
service/ai-open-webui-service-01   ClusterIP   172.30.198.88   <none>        8080/TCP   18h
service/housley-counter            ClusterIP   172.30.203.60   <none>        8080/TCP   3h44m

NAME                                          READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/ai-open-webui-deployment-01   1/1     1            1           18h
deployment.apps/housley-counter               1/1     1            1           3h44m

NAME                                                     DESIRED   CURRENT   READY   AGE
replicaset.apps/ai-open-webui-deployment-01-86bd8c545f   1         1         1       18h
replicaset.apps/housley-counter-58f474597f               0         0         0       3h44m
replicaset.apps/housley-counter-69fffdf548               1         1         1       3h44m

NAME                                             TYPE     FROM   LATEST
buildconfig.build.openshift.io/housley-counter   Source   Git    1

NAME                                         TYPE     FROM          STATUS     STARTED       DURATION
build.build.openshift.io/housley-counter-1   Source   Git@4fd1ad2   Complete   4 hours ago   18s

NAME                                             IMAGE REPOSITORY                                                          TAGS     UPDATED
imagestream.image.openshift.io/housley-counter   image-registry.openshift-image-registry.svc:5000/ai-s01/housley-counter   latest   4 hours ago

NAME                                          HOST/PORT                                        PATH   SERVICES                   PORT       TERMINATION     WILDCARD
route.route.openshift.io/housley-counter      housley-counter-ai-s01.apps.ocp.ucsx.hl.dns             housley-counter            8080-tcp   edge/Redirect   None
route.route.openshift.io/openwebui-route-01   openwebui-route-01-ai-s01.apps.ocp.ucsx.hl.dns          ai-open-webui-service-01   8080       edge            None
```

---


Now we will look at the output with just the names

In [ ]:
### Look at all resources in the project with options ###
!oc get all -o name


## Feel free to add another code cell and copy the above command
## Then change the -o option to see how output varies

#### Expected Output 
The output will be different to the following 


---

```
Warning: apps.openshift.io/v1 DeploymentConfig is deprecated in v4.14+, unavailable in v4.10000+
pod/ai-open-webui-deployment-01-86bd8c545f-4rv8b
pod/housley-counter-1-build
pod/housley-counter-69fffdf548-7m5cp
service/ai-open-webui-service-01
service/housley-counter
deployment.apps/ai-open-webui-deployment-01
deployment.apps/housley-counter
replicaset.apps/ai-open-webui-deployment-01-86bd8c545f
replicaset.apps/housley-counter-58f474597f
replicaset.apps/housley-counter-69fffdf548
buildconfig.build.openshift.io/housley-counter
build.build.openshift.io/housley-counter-1
imagestream.image.openshift.io/housley-counter
route.route.openshift.io/housley-counter
route.route.openshift.io/openwebui-route-01
Error from server (Forbidden): applications.app.k8s.io is forbidden: User "s01" cannot list resource "applications" in API group "app.k8s.io" in the namespace "ai-s01"

```

---


### Get details of specific object
When you query a specific object you will need to provide the object's unique name

This can be seen from the above "name" output 

we will look at a specific pod

In [ ]:
### Describe the Pod ###
# Note: The pod name will be unique for your environment, so you will need to adjust the command accordingly.
# change the xxxx below to reflect your pod number

!oc describe pod housley-counter-xxxx

#### Expected Output 
The output should look similar to 

---

```
Conditions:
  Type                        Status
  PodReadyToStartContainers   True 
  Initialized                 True 
  Ready                       True 
  ContainersReady             True 
  PodScheduled                True 
Volumes:
  kube-api-access-4sm4j:
    Type:                    Projected (a volume that contains injected data from multiple sources)
    TokenExpirationSeconds:  3607
    ConfigMapName:           kube-root-ca.crt
    ConfigMapOptional:       <nil>
    DownwardAPI:             true
    ConfigMapName:           openshift-service-ca.crt
    ConfigMapOptional:       <nil>
QoS Class:                   BestEffort
Node-Selectors:              <none>
Tolerations:                 node.kubernetes.io/not-ready:NoExecute op=Exists for 300s
                             node.kubernetes.io/unreachable:NoExecute op=Exists for 300s
Events:                      <none>
```

---


#### Look at the pod/container log files

In [ ]:
### Look at the pod logs ###
# Note: The pod name may vary, so you will need to adjust the command accordingly.
# ensure the xxxx is changed

!oc logs housley-counter-xxxx

# Again adjust name to match your environment 
# You can also use the -f option to follow the logs in real-time

#### Expected Output 
- Here you can see the output of the primary process in the pod, which is the Flask application running on port 8080.
- The output should look similar to:

---

```

---> Running application from Python script (app.py) ...
 * Serving Flask app 'app'
 * Debug mode: off
WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://10.129.3.70:8080
Press CTRL+C to quit
10.128.2.2 - - [01/Aug/2025 04:15:19] "GET / HTTP/1.1" 200 -
10.128.2.2 - - [01/Aug/2025 04:15:19] "GET /static/css/style.css HTTP/1.1" 200 -
10.128.2.2 - - [01/Aug/2025 04:15:19] "GET /static/images/housley_logo.png HTTP/1.1" 200 -
10.128.2.2 - - [01/Aug/2025 04:15:19] "GET /static/js/script.js HTTP/1.1" 200 -
10.128.2.2 - - [01/Aug/2025 04:57:11] "GET / HTTP/1.1" 200 -
```

---


### Access a different project
- If you need to access a different project, you can use add **-n project_name**

In [ ]:
!oc -n ai-class get all

#### Expected Output 
The output should look similar to 

---

```
Warning: apps.openshift.io/v1 DeploymentConfig is deprecated in v4.14+, unavailable in v4.10000+
NAME                                                          READY   STATUS    RESTARTS   AGE
pod/ai-ollama-685df76698-t6mdj                                1/1     Running   0          18h
pod/ai-workbench-0                                            2/2     Running   0          18h
pod/nim-service-meta-llama-3-2-3b-instruct-786cfc8cb7-h697f   1/1     Running   0          18h

NAME                                             TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)     AGE
service/ai-ollama-service                        ClusterIP   172.30.213.106   <none>        11434/TCP   18h
service/ai-workbench                             ClusterIP   172.30.189.240   <none>        80/TCP      18h
service/ai-workbench-tls                         ClusterIP   172.30.248.119   <none>        443/TCP     18h
service/nim-service-meta-llama-3-2-3b-instruct   ClusterIP   172.30.40.34     <none>        8000/TCP    18h

NAME                                                     READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/ai-ollama                                1/1     1            1           18h
deployment.apps/nim-service-meta-llama-3-2-3b-instruct   1/1     1            1           18h

NAME                                                                DESIRED   CURRENT   READY   AGE
replicaset.apps/ai-ollama-685df76698                                1         1         1       18h
replicaset.apps/nim-service-meta-llama-3-2-3b-instruct-786cfc8cb7   1         1         1       18h

NAME                            READY   AGE
statefulset.apps/ai-workbench   1/1     18h

NAME                                                      HOST/PORT                                                      PATH   SERVICES                                 PORT          TERMINATION          WILDCARD
route.route.openshift.io/ai-ollama-route                  ai-ollama-route-ai-class.apps.ocp.ucsx.hl.dns                         ai-ollama-service                        11434         edge                 None
route.route.openshift.io/ai-workbench                     ai-workbench-ai-class.apps.ocp.ucsx.hl.dns                            ai-workbench-tls                         oauth-proxy   reencrypt/Redirect   None
route.route.openshift.io/nim-meta-llama-3-2-3b-instruct   nim-meta-llama-3-2-3b-instruct-ai-class.apps.ocp.ucsx.hl.dns          nim-service-meta-llama-3-2-3b-instruct   8000          edge                 None
Error from server (Forbidden): applications.app.k8s.io is forbidden: User "s01" cannot list resource "applications" in API group "app.k8s.io" in the namespace "ai-class"
```

---


Continue to Lab Part D